<a href="https://colab.research.google.com/github/kiki-glitch/DSA-Tutorials-Python/blob/main/Dynamic_Arrays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import annotations
from typing import Generic, Iterator, List, Optional, TypeVar

In [ ]:
T = TypeVar('T')

In [ ]:
class DynamicArray(Generic[T]):
  """
    A generic dynamic array implementation in Python.
    - Starts with a given capacity (default 16)
    - Doubles capacity on growth when full
    - Shrinks only when removing via remove_at (to len-1 as per the Java code)
    """

  def __init__(self, capacity: int = 16) -> None:
    if capacity < 0: raise ValueError(f"Illegal Capacity: {capacity}")
    self._capacity: int = capacity
    self._len: int = 0
    self._arr: List[Optional[T]] = [None] * capacity

  #--- size & empitness ---
  def size(self) -> int:
    return self._len

  def is_empty(self) -> bool:
    if len == 0: return True
    return False

  def __len__(self) -> int:
    self._len

  #--- index access ---
  def _check_bounds(self, index: int) -> None:
    if index < 0 or index >= self._len:
      raise IndexError(f"Index ut of bounds")

  def get(self, index:int) -> T:
    self._check_bounds(index)
    return self._arr[index] #type: ignore[return-value]

  def __getitem__(self, index:int) -> T:
    return self.get(index)

  def set(self, index:int, elem: T) -> None:
    self._check_bounds(index)
    self._arr[index] = elem #type: ignore[return-value]

  def __setitem__(self, index:int, elem:T) -> None:
    return self.set(index, elem)

  #clear
  def clear()

